<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/hydrafloods/training_materials/oct_2021_hf_training/notebooks/flood_mapping_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flood mapping using HYDRAFloods

In this notebook we will look at further use of HYDRAFloods for creating flood maps. We will focus on the use of Sentinel 1 SAR data for creating flood maps for a use case for Hurricanes Eta/Iota. While we focus on SAR for this exercise, the concepts extend to optical data.

HYDRAFloods Python package can be found at: https://servir-mekong.github.io/hydra-floods/

## Setup
Before running the notebook, please mount your Google Drive to the notebook. We will use Google Drive to securely store Earth Engine credentials for use in other notebooks. This will allow us to bypass authenticating everytime saving time throughout the training.

In [ ]:
# mount the google drive so that we can save credentials
from google.colab import drive
drive.mount('/content/drive')

Now we will install the `hydrafloods` package for surface water mapping and `geemap` for interactive viewing results from Earth Engine.

You will get and error stating "*You must restart the runtime in order to use newly installed versions.*" This can be ignored.

In [ ]:
# install the packages needed
!pip install hydrafloods geemap

In [ ]:
import ee
import datetime
import hydrafloods as hf
import geemap.eefolium as geemap
import geemap.colormaps as cm

In [ ]:
_ = geemap.Map()

## Recap of mapping surface water
The start of any process is to acquire data. Here HYDRAFloods is used to connect to Earth Engine collections and apply spatio-temporal filters of our interest with minimal amount of coding.

In [ ]:
region = ee.FeatureCollection([
    hf.country_bbox("Belize"),
    hf.country_bbox("Guatemala"),
    hf.country_bbox("Honduras"),
    hf.country_bbox("El Salvador"),
    hf.country_bbox("Nicaragua")
])

In [ ]:
s1 = hf.Sentinel1(region,"2020-11-06","2020-11-07")

In [ ]:
s1.n_images

In [ ]:
merit = ee.Image("MERIT/Hydro/v1_0_1")

# extract out the DEM and HAND bands
dem = merit.select("elv").unmask(0)
hand = merit.select("hnd").unmask(0)

In [ ]:
mydem = ee.Image("NASA/NASADEM_HGT/001")

In [ ]:
# apply a (psuedo-) terrain flattening algorithm to S1 data
s1_flat = s1.apply_func(hf.slope_correction, elevation = dem, buffer = 100)

In [ ]:
# apply a speckle filter algorithm to S1 data
s1_filtered = s1_flat.apply_func(hf.gamma_map)

In [ ]:
# aggregate SAR observations to 30x30 m pixels
s1_aggregated = s1_filtered.apply_func(lambda x: x.focal_mean(40,"circle","meters").reproject(ee.Projection("EPSG:4326").atScale(30)))

In [ ]:
sar_vis = {
    "bands":"VV",
    "min":-25,
    "max":0
}

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(region,{},"Region of Interest")
Map.addLayer(s1_aggregated.collection.mosaic(),sar_vis, 'Sentinel 1 mosaic')

Map.addLayerControl()
Map

In [ ]:
# apply a water thresholding algorithm to the collection
# method from Markert et al., 2020 (https://doi.org/10.3390/rs12152469)
water = s1_filtered.apply_func(hf.edge_otsu,initial_threshold=-18,band="VH",edge_buffer=300,scale=180)

In [ ]:
water_img = water.collection.mosaic()

In [ ]:
water_final = (
    hf.close_binary(
        hf.open_binary(water_img,window=1.5) # apply opening filter
        .updateMask(water_img.mask()) # force mask to be consistent with sar imagery
        ,window=1.5
    ) # apply closing filter
    .updateMask(water_img.mask()) # force mask to be consistent with sar imagery
    .And(hand.lt(15)) # only pixels that were originally classified as water AND < 15m from HAND
)

Note: it is on the to-do list to make the "cleaning" of water imagery more intutive for users

In [ ]:
# view the results of SAR water mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(s1_aggregated.collection.median(),{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_final.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")

Map.addLayerControl()
Map

In [ ]:
#@title Export to asset example
hf.export_image(
    s1_aggregated.collection.mosaic().addBands(water_final),
    region.geometry(),
    description = "hydrafloods_training_day2_water",
    scale=30,
    crs='EPSG:4326',
    pyramiding={"water":"mode"},
    export_type='toAsset',
    asset_id = "users/kelmarkert/public/hydrafloods_training_day2_water"
)


In [ ]:
hf.export_image(
    water_final,
    region.geometry(),
    description = "hydrafloods_training_day2_water_export_live",
    scale=30,
    crs='EPSG:4326',
    pyramiding={".default":"mode"},
    export_type='toDrive'
)


## Extracting floods from permanent water

To extract floods we need some indicator of what is normal. The "normal" usually influence what the resulting flood map looks like. In these following cases we will use the [JRC surface water data](https://doi.org/10.1038/nature20584) to compare the event against and extract the flooded areas.

In [ ]:
# start by pulling in a pre-computed water layer from earlier
event_img = ee.Image("users/kelmarkert/public/hydrafloods_training_day2_water")

sar_img = event_img.select("V.*")
water_img = event_img.select("water")

### Using yearly permanent water

For this case we will use previous 5-years of classified permanent water to compare against.

In [ ]:
water_img.date().format("YYYY-MM-dd").getInfo()

In [ ]:
# use `extract_flood` function to use JRC data as reference
floods_yearly = hf.extract_flood(water_img,reference="yearly")

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")
Map.addLayer(floods_yearly.selfMask(),{"min":0,"max":1,"palette":"red"}, "Sentinel 1 (flood)")

Map.addLayerControl()
Map

### Using long-term occurrence

Here we use the full record of JRC observed water and define permanent using a % occurrence threshold.

In [ ]:
# use `extract_flood` function to use JRC data as reference
floods_occurrence = hf.extract_flood(water_img,reference="occurrence",permanent_threshold=50)

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")
Map.addLayer(floods_occurrence.selfMask(),{"min":0,"max":1,"palette":"red"}, "Sentinel 1 (flood)")

Map.addLayerControl()
Map

### Using seasonal information

Here we will use the occurrence for the month that we observe to remove any effects of seasonality that is present in floods.

In [ ]:
# use `extract_flood` function to use JRC data as reference
floods_seasonal = hf.extract_flood(water_img,reference="seasonal",permanent_threshold=70)

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")
Map.addLayer(floods_seasonal.selfMask(),{"min":0,"max":1,"palette":"red"}, "Sentinel 1 (flood)")

Map.addLayerControl()
Map

### Comparing the different approaches

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(s1_aggregated.collection.median(),{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_final.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")
Map.addLayer(floods_yearly.selfMask(),{"min":0,"max":1,"palette":"#FF7F7F"}, "Sentinel 1 (yearly)")
Map.addLayer(floods_occurrence.selfMask(),{"min":0,"max":1,"palette":"red"}, "Sentinel 1 (ocurrence)")
Map.addLayer(floods_seasonal.selfMask(),{"min":0,"max":1,"palette":"darkred"}, "Sentinel 1 (seasonal)")

Map.addLayerControl()
Map

Any caveauts to using these methods?

## Pre- and post-event differencing



In [ ]:
# get imagery for a period before event
preevent = hf.Sentinel1Asc(region,"2020-09-01","2020-10-01")

In [ ]:
# apply a (psuedo-) terrain flattening algorithm to S1 data
preevent_flat = preevent.apply_func(hf.slope_correction, elevation = dem, buffer = 100)

In [ ]:
# apply a speckle filter algorithm to S1 data
preevent_filtered = preevent_flat.apply_func(hf.gamma_map)

In [ ]:
pre_img = preevent_filtered.collection.mean()

### Classification differencing

In [ ]:
pre_water = hf.edge_otsu(pre_img,band="VV",initial_threshold=-16,edge_buffer=300,region=region,scale=300)

In [ ]:
floods_prepost = hf.discrete_difference(water_img,pre_water)

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(pre_img,{"bands": "VV", "min":-25, "max": 0}, 'pre-event Sentinel 1')
Map.addLayer(pre_water.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "pre-event water")
Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, "post-event Sentinel 1")
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":"lightblue"}, "post-event water")
Map.addLayer(floods_prepost.selfMask(),{"min":0,"max":1,"palette":"red"}, "event flooding")

Map.addLayerControl()
Map

### Hurricane Mitch

In [ ]:
lt5 = hf.Landsat5(region,"1998-09-01","1998-10-01")

In [ ]:
mndwi = lt5.apply_func(hf.mndwi)

In [ ]:
mitch_water = mndwi.apply_func(hf.edge_otsu,band="mndwi",initial_threshold=0,edge_buffer=300,scale=300,invert=True)

In [ ]:
floods_mitch = hf.discrete_difference(water_img,mitch_water.collection.mode())

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(lt5.collection.mean(),{"bands": "swir2,nir,green", "min":50, "max": 5500,"gamma":1.5}, 'Landsat 5 Mitch')
Map.addLayer(mitch_water.collection.mode().selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "pre-event water")
Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, "post-event Sentinel 1")
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":"lightblue"}, "post-event water")
Map.addLayer(floods_prepost.selfMask(),{"min":0,"max":1,"palette":"red"}, "event flooding")

Map.addLayerControl()
Map

### Change detection

In this case we we will use some change detection techniques to identify changes between a reference (pre-event) and post-event imagery. A famous algorithm for SAR is the [logarithmic amplitude ratio (LAR)](https://doi.org/10.1080/014311698215649). Here we will compute the LAR for a pair of pre- and post-event imagery and extract floods.

In [ ]:
# convert the db data to amplitude power
# then divide post/pre and take the log
lar = hf.db_to_power(sar_img.select("VV")).divide(hf.db_to_power(pre_img.select("VV"))).log10()

In [ ]:
floods_lar = hf.edge_otsu(lar,region=region,initial_threshold=0,scale=300,edge_buffer=300)

In [ ]:
floods_lar_final = (
    hf.close_binary(
        hf.open_binary(floods_lar,window=1.5) # apply opening filter
        .updateMask(floods_lar.mask()) # force mask to be consistent with sar imagery
        ,window=1.5
    ) # apply closing filter
    .updateMask(floods_lar.mask()) # force mask to be consistent with sar imagery
    .And(hand.lt(15)) # only pixels that were originally classified as water AND < 15m from HAND
)

In [ ]:
# view the results of flood mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(pre_img,{"bands": "VV", "min":-25, "max": 0}, 'pre-event Sentinel 1')
Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, "post-event Sentinel 1")
Map.addLayer(lar,{"min":-1,"max":1,"palette":cm.palettes.inferno}, "LAR")
Map.addLayer(floods_lar_final.selfMask(),{"min":0,"max":1,"palette":"red"}, "LAR event flooding")


Map.addLayerControl()
Map

## Flood depth estimation

Sometimes not only extent is needed and some estimate of depth provides a better estimate of impacts from floods. `hydrafloods` has the [Floodwater Depth Estimation Tool (FwDET)](https://doi.org/10.5194/nhess-19-2053-2019) implemented to provide an estimate of flood depth based off of a DEM.

**CAUTION**: This algorithm provides a fairly rough estimate of flood depth and its accuracy is very dependant on the accuracy of the input DEM.

In [ ]:
# apply the fwdet algorithm for the floods we extracted
flood_depths = hf.fwdet(water_img,dem,force_projection=True)

In [ ]:
# view the results of flood depth mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(sar_img,{"bands": "VV", "min":-25, "max": 0}, 'Sentinel 1')
Map.addLayer(water_img.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, "Sentinel 1 (water)")
Map.addLayer(floods_yearly.selfMask(),{"min":0,"max":1,"palette":"red"}, "Sentinel 1 (flood)")
Map.addLayer(flood_depths,{"bands":"depth","max":5,"palette":cm.palettes.viridis_r},"Flood depth")


Map.addLayerControl()
Map

## Creating static maps

In [ ]:
!pip install cartopy scipy

In [ ]:
from geemap import cartoee
import cartopy.crs as ccrs

In [ ]:
ax = cartoee.get_map(sar_img, region=[-90.6530,15.9700,-90.3825,16.0676],)